## Get Housing data from Burlington, VT's Open Data Repo

https://maps.burlingtonvt.gov/portal/apps/sites/#/open-data/datasets/604e666d05e9496a82c524c9e027a4cb/data?orderBy=objectid&orderByAsc=false

Example:
```
URL = "https://maps.burlingtonvt.gov/arcgis/rest/services/Hosted/Assessor_Property_Building_Information/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json"
burlington_property = requests.get(URL)  

burlington_property.json()['exceededTransferLimit'] # if this is true, there are more pages

burlington_property.json().keys()
# Returns: dict_keys(['exceededTransferLimit', 'features', 'fields', 'objectIdFieldName', 'hasZ', 'hasM'])

# get the last value from the data pulled:
pd.json_normalize(burlington_property.json()['features']).iloc[-1]['attributes.objectid']
```

In [1]:
import requests
import pandas as pd

In [2]:
start_value = 1
ending_value = 2001
property_df_list = []
new_results = True


while new_results:
    URL = f"https://maps.burlingtonvt.gov/arcgis/rest/services/Hosted/Assessor_Property_Building_Information/FeatureServer/0/query?where=objectid%20%3E%3D%20{start_value}%20AND%20objectid%20%3C%3D%20{ending_value}&outFields=*&outSR=4326&f=json"
    burlington_property_raw = requests.get(URL) 
    property_df_list.append(pd.json_normalize(burlington_property_raw.json()['features']))
    new_results = burlington_property_raw.json()['exceededTransferLimit']
    start_value = pd.json_normalize(burlington_property_raw.json()['features']).iloc[-1]['attributes.objectid'] +1
    ending_value = start_value + 2001
    

In [3]:
burlington_vt_housing_data = pd.concat(property_df_list)

In [4]:
burlington_vt_housing_data.head()

,attributes.objectid,attributes.propertyid,attributes.taxparcelid,attributes.vermontspan,attributes.streetaddressformatted,attributes.primarylandusecode,attributes.primarylandusedescription,attributes.lastsaledate,attributes.lastsaleamount,attributes.lotsqfeet,...,attributes.fullbaths,attributes.halfbaths,attributes.foundationtype,attributes.heatfuel,attributes.heattype,attributes.residentialunits,attributes.commercialunits,attributes.quality,attributes.physicalcondition,attributes.depreciationpct
0,1,1,002-2-002-000,114-035-10001,820 NORTH AVE,E,Exempt,NaN,NaN,12087040.0,...,0,2,S - SLAB,G - GAS,HA - HOT AIR,N/A,1,A - AVERAGE,Average,30.0
1,2,1,002-2-002-000,114-035-10001,820 NORTH AVE,E,Exempt,NaN,NaN,12087040.0,...,1,0,S - SLAB,G - GAS,HA - HOT AIR,1,N/A,A - AVERAGE,Fair,43.0
2,3,1,002-2-002-000,114-035-10001,820 NORTH AVE,E,Exempt,NaN,NaN,12087040.0,...,1,2,S - SLAB,G - GAS,HA - HOT AIR,N/A,1,A - AVERAGE,Average,30.0
3,4,1,002-2-002-000,114-035-10001,820 NORTH AVE,E,Exempt,NaN,NaN,12087040.0,...,0,0,S - SLAB,G - GAS,HA - HOT AIR,1,N/A,A - AVERAGE,Very Good,29.6
4,5,2,003-2-001-000,114-035-10002,0 INTERVALE RD,EL,Exmpt Land,NaN,NaN,2756310.0,...,0,0,-,-,-,N/A,N/A,-,Average,0.0


In [6]:
burlington_vt_housing_data.shape

(11412, 37)